In [1]:
%run common.py

Planning start time: Jul 20, 14:07 (1469012839)


In [2]:
st_name = 'ИРКУТСК-СОРТИРОВОЧНЫЙ'

In [21]:
d = datetime.datetime.fromtimestamp(current_time - 18 * 3600)
day_start = int(datetime.datetime.timestamp(datetime.datetime(d.year, d.month, d.day, 18, 0, 0))) + 24 * 3600
print(time.ctime(current_time))
print(time.ctime(day_start))

Wed Jul 20 14:07:19 2016
Wed Jul 20 18:00:00 2016


In [22]:
team_plan['dt_start'] = team_plan.time_start.apply(datetime.datetime.fromtimestamp)
team_plan['depot_name'] = team_plan.depot.map(st_names.name)
team_start_stations = team_plan[team_plan.state.isin([0, 1])].drop_duplicates('team').set_index('team')
team_plan['start_st'] = team_plan.team.map(team_start_stations.st_from_name)
team_plan.depot.fillna(team_plan.start_st, inplace=True)

In [28]:
ct = datetime.datetime.fromtimestamp(current_time)
dt = datetime.datetime.fromtimestamp(day_start)

In [29]:
team_plan['hour_start'] = team_plan.time_start.apply(lambda x: 3600 * int(x / 3600))
team_plan['hour_start_norm'] = team_plan.hour_start.apply(nice_time)

In [39]:
# Total team depart
team_cols = ['team', 'st_from_name', 'st_to_name', 'time_start_norm', 'hour_start_norm', 'state', 'loco', 'depot_name']
a = team_plan[(team_plan.st_from_name == st_name) 
              & (team_plan.state.isin([0, 1]))
              & (team_plan.time_start >= day_start)
              & (team_plan.time_start < day_start + 24 * 3600)].sort_values('time_start')
print(a.groupby(['st_to_name', 'state']).team.count())
b = a.groupby(['st_to_name', 'hour_start_norm']).team.count().to_frame().reset_index()
c = b.pivot('st_to_name', 'hour_start_norm', 'team').fillna(0)
c

st_to_name       state
БАТАРЕЙНАЯ       0        22
                 1        84
ВОЕННЫЙ ГОРОДОК  0        40
                 1         2
ГОНЧАРОВО        0         1
                 1        40
Name: team, dtype: int64


hour_start_norm,"Jul 20, 18:00","Jul 20, 19:00","Jul 20, 20:00","Jul 20, 21:00","Jul 20, 22:00","Jul 20, 23:00","Jul 21, 00:00","Jul 21, 01:00","Jul 21, 02:00","Jul 21, 03:00",...,"Jul 21, 08:00","Jul 21, 09:00","Jul 21, 10:00","Jul 21, 11:00","Jul 21, 12:00","Jul 21, 13:00","Jul 21, 14:00","Jul 21, 15:00","Jul 21, 16:00","Jul 21, 17:00"
st_to_name,,,,,,,,,,,,,,,,,,,,,
БАТАРЕЙНАЯ,4,5,7,7,5,3,5,3,7,2,...,3,2,4,5,2,4,6,0,3,1
ВОЕННЫЙ ГОРОДОК,3,0,0,5,0,3,0,6,2,4,...,7,0,0,0,1,0,0,0,1,0
ГОНЧАРОВО,1,3,1,5,2,5,0,1,4,0,...,3,3,1,0,0,1,2,1,1,2


In [37]:
b_dep = a.groupby(['depot_name', 'hour_start_norm']).team.count().to_frame().reset_index()
c_dep = b_dep.pivot('depot_name', 'hour_start_norm', 'team').fillna(0)
print(c_dep.sum(axis = 1).sort_values(ascending=False))
c_dep

depot_name
ЗИМА                     65
ИРКУТСК-СОРТИРОВОЧНЫЙ    53
НИЖНЕУДИНСК              19
СЛЮДЯНКА I               14
УЛАН-УДЭ                  5
ТАЙШЕТ                    3
ИЛАНСКАЯ                  1
dtype: float64


hour_start_norm,"Jul 20, 18:00","Jul 20, 19:00","Jul 20, 20:00","Jul 20, 21:00","Jul 20, 22:00","Jul 20, 23:00","Jul 21, 00:00","Jul 21, 01:00","Jul 21, 02:00","Jul 21, 03:00",...,"Jul 21, 08:00","Jul 21, 09:00","Jul 21, 10:00","Jul 21, 11:00","Jul 21, 12:00","Jul 21, 13:00","Jul 21, 14:00","Jul 21, 15:00","Jul 21, 16:00","Jul 21, 17:00"
depot_name,,,,,,,,,,,,,,,,,,,,,
ЗИМА,3,1,2,4,2,2,4,2,3,0,...,4,5,5,3,2,2,1,1,1,2
ИЛАНСКАЯ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ИРКУТСК-СОРТИРОВОЧНЫЙ,3,1,3,5,4,7,1,6,3,4,...,6,0,0,0,0,0,0,0,1,0
НИЖНЕУДИНСК,0,2,1,4,0,0,0,0,0,0,...,0,0,0,1,0,1,4,0,0,0
СЛЮДЯНКА I,0,0,0,1,0,0,0,1,2,0,...,3,0,0,0,1,0,1,0,0,1
ТАЙШЕТ,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
УЛАН-УДЭ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0


In [33]:
st1 = 'БАТАРЕЙНАЯ'
d = team_plan[(team_plan.st_from_name == st_name) & (team_plan.st_to_name == st1)
           & (team_plan.state.isin([0, 1]))
           & (team_plan.time_start >= day_start)
           & (team_plan.time_start < day_start + 24 * 3600)].sort_values('time_start')
d[d.state == 1].hour_start_norm.value_counts().sort_index()
#d[d.state == 1][team_cols].head(10)

Jul 20, 18:00    1
Jul 20, 19:00    4
Jul 20, 20:00    7
Jul 20, 21:00    6
Jul 20, 22:00    5
Jul 20, 23:00    3
Jul 21, 00:00    5
Jul 21, 01:00    2
Jul 21, 02:00    7
Jul 21, 03:00    2
Jul 21, 04:00    6
Jul 21, 05:00    5
Jul 21, 06:00    4
Jul 21, 07:00    2
Jul 21, 08:00    3
Jul 21, 09:00    2
Jul 21, 10:00    4
Jul 21, 11:00    4
Jul 21, 12:00    2
Jul 21, 13:00    3
Jul 21, 14:00    5
Jul 21, 16:00    1
Jul 21, 17:00    1
Name: hour_start_norm, dtype: int64

In [8]:
team_plan['type'] = team_plan.team.apply(lambda x: int(str(x)[:1]))
print(time.ctime(current_time))
team_plan[(team_plan['type'] == 7) & (team_plan.state.isin([0, 1]) & (team_plan.time_start <= current_time + 30 * 3600))]\
        .drop_duplicates('team').hour_start_norm.value_counts().sort_index()

Wed Jul 20 14:07:19 2016


Jul 20, 14:00     18
Jul 20, 15:00    141
Jul 20, 16:00    110
Jul 20, 17:00    115
Jul 20, 18:00     44
Jul 20, 19:00    180
Jul 20, 20:00     30
Jul 20, 21:00    105
Jul 20, 22:00     27
Jul 20, 23:00     32
Jul 21, 00:00     36
Jul 21, 01:00     39
Jul 21, 02:00     31
Jul 21, 03:00     45
Jul 21, 04:00     33
Jul 21, 05:00     30
Jul 21, 06:00     32
Jul 21, 07:00     20
Jul 21, 08:00     28
Jul 21, 09:00     37
Jul 21, 10:00     53
Jul 21, 11:00     33
Jul 21, 12:00     34
Jul 21, 13:00     25
Jul 21, 14:00     28
Jul 21, 15:00     23
Jul 21, 16:00     57
Jul 21, 17:00     43
Jul 21, 18:00     44
Jul 21, 19:00     41
Jul 21, 20:00      4
Name: hour_start_norm, dtype: int64

In [9]:
st_name = 'ИРКУТСК-СОРТИРОВОЧНЫЙ'
irk_t = team_plan[(team_plan.st_from_name == st_name) & (team_plan.state.isin([0, 1]))
         & (team_plan.time_start >= current_time) & (team_plan.time_start < current_time + 24 * 3600)]
irk = team_plan[(team_plan.team.isin(irk_t.team)) & (team_plan.state.isin([0, 1]))].copy(deep=True)
mins = irk.groupby('team').time_start.min()
irk['end_trip'] = (irk.team != irk.team.shift(-1)) | (irk.state != irk.state.shift(-1)) | (irk.loco != irk.loco.shift(-1))
irk['start_trip'] = (irk.team != irk.team.shift(1)) | (irk.state != irk.state.shift(1)) | (irk.loco != irk.loco.shift(1))
irk[['team', 'start_trip', 'end_trip']]
irk[(irk.start_trip == True) | (irk.end_trip == True)][['team', 'st_from_name', 'st_to_name', 'state', 'loco']]
starts = irk[(irk.start_trip == True)][['team', 'st_from_name', 'time_start', 'time_start_norm', 'state', 'loco']].reset_index()
ends = irk[(irk.end_trip == True)][['st_to_name', 'time_end', 'time_end_norm']].reset_index()
cols = ['team', 'st_from_name', 'st_to_name', 'time_start', 'time_end', 'time_start_norm', 'time_end_norm', 'state', 'loco']
trips = pd.concat([starts, ends], axis=1)[cols]
trips[(trips.st_from_name == st_name)].st_to_name.value_counts()

ЗИМА                     82
СЛЮДЯНКА I               61
СУХОВСКАЯ-ЮЖНАЯ           6
СУХОВСКАЯ                 6
ЧЕРЕМХОВО                 6
НИЖНЕУДИНСК               6
КИТОЙ                     4
КАСЬЯНОВКА                3
ИРКУТСК-СОРТИРОВОЧНЫЙ     3
БОЛЬШОЙ ЛУГ               3
КАЯ                       3
МЕГЕТ                     2
МЫСОВАЯ                   1
АНГАРСК                   1
АНДРИАНОВСКАЯ             1
Name: st_to_name, dtype: int64